# Preprocessing

In this notebook I'll gather and preprocess the Eurovision audience vote data used in the [main notebook](eurovision.ipynb).

In [1]:
import json

import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
import xarray as xr

## Votes

Obtained from [this project on Github](https://github.com/Spijkervet/eurovision-dataset/).

In [2]:
votes = pd.read_csv(
    "https://github.com/Spijkervet/eurovision-dataset/releases/download/2023/votes.csv"
)
votes.head()

,year,round,from_country_id,to_country_id,from_country,to_country,total_points,tele_points,jury_points
0,1957,final,at,nl,at,nl,6,NaN,NaN
1,1957,final,at,fr,at,fr,0,NaN,NaN
2,1957,final,at,dk,at,dk,0,NaN,NaN
3,1957,final,at,lu,at,lu,3,NaN,NaN
4,1957,final,at,de,at,de,0,NaN,NaN


We are only interested in the total points in the final, from 1998 onward, and want to use uppercase ISO 3166 codes to identify the countries.

In [3]:
votes = votes.loc[(votes["year"] >= 1998) & (votes["round"] == "final")]
votes["from_country_id"] = votes["from_country_id"].str.upper()
votes["to_country_id"] = votes["to_country_id"].str.upper()
votes["points"] = votes.apply(
    lambda r: r["total_points"] if pd.isna(r["tele_points"]) else r["tele_points"],
    axis=1,
)
votes = votes[["year", "from_country_id", "to_country_id", "points"]]
# filter out 'world', Australia & 'CS'
for region in ("WLD", "AU", "CS"):
    votes = votes.loc[votes["from_country_id"] != region]
    votes = votes.loc[votes["to_country_id"] != region]

votes = votes.sort_values(by=["year", "from_country_id", "points"]).reset_index(
    drop=True
)
votes["points_encoded"] = OrdinalEncoder().fit_transform(
    votes["points"].to_numpy().reshape(-1, 1)
)
votes["points_encoded"] = votes["points_encoded"].astype(int)
votes["points"] = votes["points"].astype(int)
votes.head()

,year,from_country_id,to_country_id,points,points_encoded
0,1998,BE,BE,0,0
1,1998,BE,IE,0,0
2,1998,BE,SE,0,0
3,1998,BE,EE,0,0
4,1998,BE,TR,0,0


In [4]:
votes.to_csv("data/votes.csv", index=False)

I'll also transform it to an xArray, for easier use later.

In [5]:
votes = votes[votes["year"] > 2014]
from_countries = sorted(votes["from_country_id"].unique())
to_countries = sorted(votes["to_country_id"].unique())
years = sorted(votes["year"].unique())

from_country_map = {country: i for i, country in enumerate(from_countries)}
to_country_map = {country: i for i, country in enumerate(to_countries)}
year_map = {year: i for i, year in enumerate(years)}

votes_array_3d = np.zeros((len(from_countries), len(to_countries), len(years)))

for _, row in votes[votes["year"] > 2014].iterrows():
    i = from_country_map[row["from_country_id"]]
    j = to_country_map[row["to_country_id"]]
    k = year_map[row["year"]]
    votes_array_3d[i, j, k] = row["points_encoded"]

vote_array = xr.DataArray(
    data=votes_array_3d,
    dims=['voter', 'performer', 'year'],
    coords={
        'voter': from_countries,
        'performer': to_countries,
        'year': years
    },
    name='points_encoded'
)
vote_array

<xarray.DataArray 'points_encoded' (voter: 43, performer: 42, year: 8)> Size: 116kB
array([[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  2.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0., ...,  0.,  0.,  2.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  6.,  0., ...,  4.,  9.,  0.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  4.,  0., ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  9.,  0., ...,  0.,  9.,  1.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  3.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ...,
...
        ...,
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  7.,  0., ...,  7.,  9.,  0.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  2.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0., 10.,  0., ...,  5., 10.,  6.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  7.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]]], shape=(43, 42, 8))
Coordinates:
  * voter      (voter) <U2 344B 'AL' 'AM' 'AT' 'AZ' 'BA' ... 'SE' 'SI' 'SM' 'UA'
  * performer  (performer) <U2 336B 'AL' 'AM' 'AT' 'AZ' ... 'SE' 'SI' 'SM' 'UA'
  * year       (year) int64 64B 2015 2016 2017 2018 2019 2021 2022 2023

In [6]:
vote_array.to_netcdf("data/votes.nc")

## Migrant stock

Data obtained from the United Nations Population division [here](https://www.un.org/development/desa/pd/content/international-migrant-stock).

In [7]:
migration = pd.read_excel(
    "data/undesa_pd_2024_ims_stock_by_sex_destination_and_origin.xlsx",
    sheet_name="Table 1",
    header=10,
    usecols="E,G,N",
)
migration.head()

,Location code of destination,Location code of origin,2020
0,900,900,275284032
1,900,1834,27134957
2,900,1833,37196853
3,900,1831,48594959
4,900,1832,38223520


This publication uses [M-49 country codes](https://en.wikipedia.org/wiki/UN_M49) - we need to translate these to ISO-3166 alpha-2 codes, for consistency with the rest of the data. I'll use [this mapping](https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes/) to convert them.

In [8]:
with open("data/slim-2.json", "r") as f:
    dict_list = json.load(f)

iso_3166_mapping = {d["country-code"]: d["alpha-2"] for d in dict_list}

migration["from_country_id"] = migration["Location code of origin"].apply(
    lambda c: iso_3166_mapping.get(str(c).zfill(3))
)
migration["to_country_id"] = migration["Location code of destination"].apply(
    lambda c: iso_3166_mapping.get(str(c).zfill(3))
)
unique_countries = sorted(
    set(votes["from_country_id"].unique()) | set(votes["to_country_id"].unique())
)
migration["eurovision"] = migration.apply(
    lambda r: r["from_country_id"] in unique_countries
    and r["to_country_id"] in unique_countries,
    axis=1,
)
migration = migration[migration["eurovision"]]
migration = (
    migration.dropna(subset=["from_country_id", "to_country_id"])
    .drop(
        columns=[
            "eurovision",
            "Location code of destination",
            "Location code of origin",
        ]
    )
    .rename(columns={2020: "migrant_stock"})
    .reset_index(drop=True)
)
migration.head()

,migrant_stock,from_country_id,to_country_id
0,84050,AZ,AM
1,34247,GE,AM
2,21099,RU,AM
3,2354,UA,AM
4,12,GR,AM


This first line, coincidentally, probably represents the population of former Nagorno-Karabakh (internationally recognized as Azerbaijani territory) moving to Armenia. As these people are ethnically Armenian, with presumably no affinity for Azerbaijan, I'll set this line to 0.

In [9]:
migration.loc[0, "migrant_stock"] = 0

I want to normalize these stocks by the total population. I'll use another [UN dataset](https://population.un.org/wpp/assets/Excel%20Files/1_Indicator%20(Standard)/EXCEL_FILES/1_General/WPP2024_GEN_F01_DEMOGRAPHIC_INDICATORS_COMPACT.xlsx).

In [10]:
population = pd.read_excel(
    "data/WPP2024_GEN_F01_DEMOGRAPHIC_INDICATORS_COMPACT.xlsx",
    sheet_name="Estimates",
    header=16,
    usecols="G,K,L",
)
population.head()

,ISO2 Alpha-code,Year,"Total Population, as of 1 January (thousands)"
0,NaN,1950.0,2471424.002
1,NaN,1951.0,2514761.693
2,NaN,1952.0,2559092.377
3,NaN,1953.0,2609080.302
4,NaN,1954.0,2659132.169


In [11]:
population = population.loc[population["Year"] == 2020.0]
population = population.rename(
    columns={
        "Total Population, as of 1 January (thousands)": "population",
        "ISO2 Alpha-code": "country_id",
    }
)
population["eurovision"] = population["country_id"].apply(lambda c: c in unique_countries)
population = population[population["eurovision"]]
population = population.drop(columns=["Year", "eurovision"]).reset_index(drop=True)
population["population"] *= 1e3
population.head()

,country_id,population
0,AM,2900489.0
1,AZ,10153702.0
2,CY,1294504.0
3,GE,3796876.0
4,IL,8730394.0


In [12]:
migration = migration.merge(
    population, how="left", left_on="to_country_id", right_on="country_id"
).drop(columns="country_id")
migration["migrant_stock_pct"] = (
    migration["migrant_stock"] / migration["population"]
)
migration["migrant_stock_pct_std"] = (
    migration["migrant_stock_pct"] - migration["migrant_stock_pct"].mean()
) / migration["migrant_stock_pct"].std()
migration.sort_values(by="migrant_stock_pct", ascending=False).head(10)

,migrant_stock,from_country_id,to_country_id,population,migrant_stock_pct,migrant_stock_pct_std
957,4749,IT,SM,34714.0,0.136804,15.598193
738,381100,BA,HR,3970038.0,0.095994,10.855149
417,119097,RU,EE,1328857.0,0.089624,10.114756
358,3330586,RU,UA,44835870.0,0.074284,8.331919
119,672481,RU,BY,9400170.0,0.071539,8.012917
855,31494,GB,MT,514526.0,0.06121,6.812382
557,116771,RU,LV,1908100.0,0.061198,6.810963
1011,122235,BA,SI,2090739.0,0.058465,6.493375
497,21102,PL,IS,364277.0,0.057928,6.431018
538,285496,GB,IE,4962558.0,0.05753,6.38471


In [13]:
migration.to_csv("data/migrants.csv", index=False)

I'll also save the pivoted data to more directly be able to use it.

In [14]:
migrants_pivoted = migration.pivot_table(
    index="from_country_id",
    columns="to_country_id",
    values="migrant_stock_pct_std",
    fill_value=migration["migrant_stock_pct_std"].min(),
)
migrants_pivoted.to_csv("data/migrants_pivoted.csv")

/tmp/ipykernel_109707/2631237400.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  migrants_pivoted = migration.pivot_table(


## Borders

In [15]:
borders = pd.DataFrame(0, index=unique_countries, columns=unique_countries)
for country in unique_countries:
    borders.loc[country, country] = 1

# Define land borders
land_borders = [
    ("AD", "ES"),  # Andorra
    ("AD", "FR"),
    ("AL", "GR"),  # Albania
    ("AL", "ME"),
    ("AL", "MK"),
    ("AM", "AZ"),  # Armenia
    ("AM", "GE"),
    ("AM", "TR"),
    ("AT", "CH"),  # Austria
    ("AT", "CZ"),
    ("AT", "DE"),
    ("AT", "HU"),
    ("AT", "IT"),
    ("AT", "SI"),
    ("AT", "SK"),
    ("AZ", "GE"),  # Azerbaijan
    ("AZ", "RU"),
    ("AZ", "TR"),
    ("AZ", "AM"),
    ("BA", "HR"),  # Bosnia and Herzegovina
    ("BA", "ME"),
    ("BA", "RS"),
    ("BE", "DE"),  # Belgium
    ("BE", "FR"),
    ("BE", "NL"),
    ("BG", "GR"),  # Bulgaria
    ("BG", "MK"),
    ("BG", "RO"),
    ("BG", "RS"),
    ("BG", "TR"),
    ("BY", "LT"),  # Belarus
    ("BY", "LV"),
    ("BY", "PL"),
    ("BY", "RU"),
    ("BY", "UA"),
    ("CH", "DE"),  # Switzerland
    ("CH", "FR"),
    ("CH", "IT"),
    ("CH", "AT"),
    ("CZ", "DE"),  # Czech Republic
    ("CZ", "PL"),
    ("CZ", "SK"),
    ("CZ", "AT"),
    ("DE", "DK"),  # Germany
    ("DE", "CH"),
    ("DE", "CZ"),
    ("DE", "FR"),
    ("DE", "NL"),
    ("DE", "PL"),
    ("DE", "AT"),
    ("DE", "BE"),
    ("EE", "LV"),  # Estonia
    ("EE", "RU"),
    ("ES", "FR"),  # Spain
    ("ES", "PT"),
    ("ES", "AD"),
    ("FI", "NO"),  # Finland
    ("FI", "RU"),
    ("FI", "SE"),
    ("FR", "IT"),  # France
    ("FR", "MC"),
    ("FR", "ES"),
    ("FR", "BE"),
    ("FR", "CH"),
    ("FR", "DE"),
    ("GB", "IE"),  # United Kingdom
    ("GE", "RU"),  # Georgia
    ("GE", "TR"),
    ("GE", "AM"),
    ("GE", "AZ"),
    ("GR", "BG"),  # Greece
    ("GR", "MK"),
    ("GR", "TR"),
    ("GR", "AL"),
    ("HR", "HU"),  # Croatia
    ("HR", "ME"),
    ("HR", "RS"),
    ("HR", "SI"),
    ("HR", "BA"),
    ("HU", "RO"),  # Hungary
    ("HU", "RS"),
    ("HU", "SI"),
    ("HU", "SK"),
    ("HU", "UA"),
    ("HU", "AT"),
    ("HU", "HR"),
    ("IT", "SI"),  # Italy
    ("IT", "SM"),
    ("IT", "AT"),
    ("IT", "FR"),
    ("IT", "CH"),
    ("LT", "LV"),  # Lithuania
    ("LT", "PL"),
    ("LT", "RU"),
    ("LT", "BY"),
    ("LV", "LT"),  # Latvia
    ("LV", "RU"),
    ("LV", "BY"),
    ("LV", "EE"),
    ("MD", "RO"),  # Moldova
    ("MD", "UA"),
    ("ME", "AL"),  # Montenegro
    ("ME", "BA"),
    ("ME", "HR"),
    ("ME", "RS"),
    ("MK", "AL"),  # North Macedonia
    ("MK", "BG"),
    ("MK", "GR"),
    ("MK", "RS"),
    ("NO", "RU"),  # Norway
    ("NO", "SE"),
    ("NO", "FI"),
    ("PL", "RU"),  # Poland
    ("PL", "SK"),
    ("PL", "UA"),
    ("PL", "BY"),
    ("PL", "CZ"),
    ("PL", "DE"),
    ("PL", "LT"),
    ("RO", "BG"),  # Romania
    ("RO", "HU"),
    ("RO", "MD"),
    ("RO", "RS"),
    ("RO", "UA"),
    ("RS", "BA"),  # Serbia
    ("RS", "BG"),
    ("RS", "HR"),
    ("RS", "HU"),
    ("RS", "ME"),
    ("RS", "MK"),
    ("RS", "RO"),
    ("RU", "BY"),  # Russia
    ("RU", "EE"),
    ("RU", "FI"),
    ("RU", "GE"),
    ("RU", "LT"),
    ("RU", "LV"),
    ("RU", "NO"),
    ("RU", "PL"),
    ("RU", "UA"),
    ("RU", "AZ"),
    ("SE", "FI"),  # Sweden
    ("SE", "NO"),
    ("SI", "AT"),  # Slovenia
    ("SI", "HR"),
    ("SI", "HU"),
    ("SI", "IT"),
    ("SK", "AT"),  # Slovakia
    ("SK", "CZ"),
    ("SK", "HU"),
    ("SK", "PL"),
    ("SK", "UA"),
    ("TR", "AM"),  # Turkey
    ("TR", "AZ"),
    ("TR", "BG"),
    ("TR", "GE"),
    ("TR", "GR"),
    ("UA", "BY"),  # Ukraine
    ("UA", "HU"),
    ("UA", "MD"),
    ("UA", "PL"),
    ("UA", "RO"),
    ("UA", "RU"),
    ("UA", "SK"),
]

# Select maritime borders
maritime_borders = [
    ("CY", "GR"),  # Cyprus - Greece
    ("GR", "CY"),
    ("DK", "SE"),  # Denmark - Sweden
    ("SE", "DK"),
    ("FI", "EE"),  # Finland - Estonia
    ("EE", "FI"),
    ("IT", "MT"),  # Malta - Italy
    ("MT", "IT"),
]

for country1, country2 in land_borders + maritime_borders:
    if country1 in unique_countries and country2 in unique_countries:
        borders.loc[country1, country2] = 1
        borders.loc[country2, country1] = 1

borders.iloc[:20, :20]

,AL,AM,AT,AZ,BA,BE,BG,BY,CH,CY,CZ,DE,DK,EE,ES,FI,FR,GB,GE,GR
AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
AM,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
AT,0,0,1,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0
AZ,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
BA,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
BE,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0
BG,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
BY,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
CH,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0
CY,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1


In [16]:
borders.to_csv("data/borders.csv")

## Songs

I'll use [yet another dataset from Github](https://github.com/josago97/EurovisionDataset).

In [17]:
with open("data/eurovision.json", "r") as f:
    eurovision_json = json.load(f)

performances = pd.DataFrame(
    [
        {
            "year": d["year"],
            "country_id": contestant["country"],
            "performer": contestant["artist"],
            "language": ", ".join(sorted(contestant["lyrics"][0]["languages"])),
            "url": contestant.get("videoUrls", [None])[0],
        }
        for d in eurovision_json
        for contestant in d["contestants"]
        if d["year"] >= 2015
        and d["year"] <= 2023
        and contestant["country"] in unique_countries
    ]
)
# filter out the performances that didn't get to the final
performances = performances.merge(
    votes[["year", "to_country_id"]].drop_duplicates(),
    how="inner",
    left_on=["year", "country_id"],
    right_on=["year", "to_country_id"],
)[["year", "country_id", "performer", "language", "url"]]
performances.head()

,year,country_id,performer,language,url
0,2015,SE,Måns Zelmerlöw,English,https://www.youtube-nocookie.com/embed/5sGOwFV...
1,2015,RU,Polina Gagarina,English,https://www.youtube.com/embed/jBVY7Glcd84
2,2015,IT,Il Volo,Italian,https://www.youtube-nocookie.com/embed/1TOMqZV...
3,2015,BE,Loïc Nottet,English,https://www.youtube-nocookie.com/embed/G48p8eN...
4,2015,LV,Aminata Savadogo,English,https://www.youtube-nocookie.com/embed/-usdXbe...


In [18]:
performances.to_csv("data/performances.csv", index=False)

I processed this file manually to get the language (in `data/languages.csv`).

## Languages

In [19]:
languages = pd.read_csv("data/languages.csv", index_col=0)
languages = languages.loc[languages.index > 2014]
languages_pivoted = languages.pivot_table(
    index="country_id",
    columns="year",
    values="language_id",
    fill_value=0,
)
languages_pivoted.iloc[:5, :5]

year,2015,2016,2017,2018,2019
country_id,,,,,
AL,0.0,0.0,0.0,1.0,1.0
AM,0.0,0.0,0.0,0.0,0.0
AT,0.0,2.0,0.0,0.0,0.0
AZ,0.0,0.0,0.0,0.0,0.0
BE,0.0,0.0,0.0,0.0,0.0


In [20]:
languages_pivoted.to_csv("data/languages_pivoted.csv")